In [2]:
!pip install pandas requests python-dotenv tqdm


In [3]:
import os
import pandas as pd
import requests
from tqdm import tqdm
import json

In [5]:
os.environ["VOID_API_KEY"] = "sk-voidai-zAFjyYrdhOfQwZahZFXpJm4U3DC4wni8BgbWyxi99B05kiIxkCitl3LjE6z09DBsCNuJnUE0JAGwcZleA4BGJOVaAnHWp6TuHQRQIRQK5EEnuHCRBTGh0hpSDT3kib_LkY19HQ"
os.environ["VOID_API_URL"] = "https://api.voidai.app/v1/chat/completions"
os.environ["VOID_MODEL"] = "gpt-4o"

API_KEY = os.getenv("VOID_API_KEY")
API_URL = os.getenv("VOID_API_URL")
MODEL = os.getenv("VOID_MODEL")

In [4]:
def call_llm(prompt):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0
    }
    response = requests.post(API_URL, json=payload, headers=headers)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]

In [6]:
from google.colab import files
uploaded = files.upload()


Saving yelp.csv to yelp.csv


In [7]:
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)
df = df.sample(200, random_state=42)
df.reset_index(drop=True, inplace=True)

In [8]:
def prompt_v1(review):
    return f"""
Classify the following Yelp review into 1–5 stars.

Review:
{review}

Return JSON:
{{
  "predicted_stars": <1-5>,
  "explanation": "short reasoning"
}}
"""

def prompt_v2(review):
    return f"""
You are a Yelp rating expert.

Example:
Review: "Amazing food and staff"
Output: {{ "predicted_stars": 5, "explanation": "Very positive sentiment" }}

Example:
Review: "Terrible service, food was cold"
Output: {{ "predicted_stars": 1, "explanation": "Strong negative sentiment" }}

Now rate:
Review: "{review}"

Return only JSON.
"""

def prompt_v3(review):
    return f"""
Rate this review using these rules:
1 = very negative
3 = neutral/mixed
5 = very positive

Review:
{review}

Respond with STRICT JSON only.
"""

prompts = [prompt_v1, prompt_v2, prompt_v3]

In [9]:
import pandas as pd

df = pd.read_csv("yelp.csv")
print(df.columns)
df.head()


Index(['business_id', 'date', 'review_id', 'stars', 'text', 'type', 'user_id',
       'cool', 'useful', 'funny'],
      dtype='object')


,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [10]:
df = df.sample(200, random_state=42)
df.reset_index(drop=True, inplace=True)

review_column = 'text'
stars_column = 'stars'

results = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    review = row[review_column]
    row_result = {"review": review}
    for j, prompt_func in enumerate(prompts, 1):
        try:
            output = call_llm(prompt_func(review))
            parsed = json.loads(output.replace("'", '"'))
        except Exception as e:
            parsed = {"predicted_stars": None, "explanation": str(e)}
        row_result[f"prompt_{j}"] = parsed
    results.append(row_result)

results_df = pd.DataFrame(results)


100%|██████████| 200/200 [07:13<00:00,  2.17s/it]


In [11]:
accuracy_table = {}
for j in range(1, 4):
    correct = 0
    total = 0
    for idx, row in results_df.iterrows():
        pred = row[f'prompt_{j}']['predicted_stars']
        if pred is None:
            continue
        total += 1
        if int(pred) == int(df.loc[idx, stars_column]):
            correct += 1
    accuracy_table[f'prompt_{j}'] = round(correct / total, 2) if total else 0

print("Accuracy per prompt:", accuracy_table)


Accuracy per prompt: {'prompt_1': 0, 'prompt_2': 0, 'prompt_3': 0}


In [12]:
comparison_df = pd.DataFrame({
    "Prompt": ["Prompt 1", "Prompt 2", "Prompt 3"],
    "Accuracy": [accuracy_table['prompt_1'], accuracy_table['prompt_2'], accuracy_table['prompt_3']],
    "JSON Validity (%)": [
        round(sum([1 for r in results_df['prompt_1'] if r['predicted_stars'] is not None])/len(results_df)*100,2),
        round(sum([1 for r in results_df['prompt_2'] if r['predicted_stars'] is not None])/len(results_df)*100,2),
        round(sum([1 for r in results_df['prompt_3'] if r['predicted_stars'] is not None])/len(results_df)*100,2)
    ]
})

comparison_df

,Prompt,Accuracy,JSON Validity (%)
0,Prompt 1,0,0.0
1,Prompt 2,0,0.0
2,Prompt 3,0,0.0
